<a href="https://colab.research.google.com/github/uday160386/ai_ml_exercises/blob/main/Group_14_Voice_E_commerce_Ordering_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

In [ ]:
#@title Explanation Video
from IPython.display import HTML

HTML("""<video width="854" and height="480" controls>
  <source src="https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Walkthrough/Hackathon_Voice_based.mp4" type="video/mp4">
</video>
""")

# Hackathon: Voice commands based E-commerce ordering system
The goal of the hackathon is to train your model on different types of voice data (such as studio data and your own team data) and able to place order based on user preferences.

## Grading = 40 Marks

### **Objectives:**

Stage 0 - Obtain Features from Audio samples

Stage 1 (22 Marks) - Define and train a CNN model on Studio data and deploy the model in the server

Stage 2 (18 Marks) - Collect your voice samples (team data) and refine the classifier trained on Studio_data. Deploy the model in the server.

## Dataset Description

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The data contains voice samples of classes - Zero, One, Two, Three, Four, Five. Each class is denoted by a numerical label from 0 to 5.

The audio files collected in a Studio dataset contain very few noise samples and all the files are in wav format.

The audio files recorded for the studio are saved with the following naming convention:

● Class Representation + user_id + sample_ID (or noise + sample_ID)

> For example: The voice sample by the user b2 recorded “Zero”, it is saved as 0_b2_35.wav. Here 35 is sample ID, 2 is the user id and ‘0’ is the label of that sample.




In [ ]:
#@title Please run the setup to download the dataset

from IPython import get_ipython
ipython = get_ipython()

notebook= "Hackathon2 - Voice E-commerce Ordering System" #name of the notebook

def setup():
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Hackathon_data/B17_studio_rev_data.zip")
    ipython.magic("sx unzip B17_studio_rev_data.zip ")
    print ("Setup completed successfully")

setup()

Setup completed successfully


In [ ]:
import os
import sys
import glob
import torch
import librosa
import warnings
import numpy as np
import torch.nn as nn
from time import sleep
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
warnings.filterwarnings('ignore')

## **Stage 0:** Obtain Features from Audio samples
---

### Generate features from an audio sample of '.wav' format
- Code is available to extract the features

In [ ]:
# Caution: Do not change the default parameters
def get_features(filepath, sr=8000, n_mfcc=30, n_mels=128, frames = 15):
    # The following function contains code to produce features of the audio sample.
    y, sr = librosa.load(filepath, sr=sr)
    D = np.abs(librosa.stft(y))**2
    S = librosa.feature.melspectrogram(S=D)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    log_S = librosa.power_to_db(S,ref=np.max)
    features = librosa.feature.mfcc(S=log_S, n_mfcc=n_mfcc)
    if features.shape[1] < frames :
        features = np.hstack((features, np.zeros((n_mfcc, frames - features.shape[1]))))
    elif features.shape[1] > frames:
        features = features[:, :frames]

    # Find 1st order delta_mfcc
    delta1_mfcc = librosa.feature.delta(features, order=1)

    # Find 2nd order delta_mfcc
    delta2_mfcc = librosa.feature.delta(features, order=2)

    # Stacking delta_mfcc features in sequence horizontally (column wise)
    features = np.hstack((delta1_mfcc.flatten(), delta2_mfcc.flatten()))

    # Increase the dimension by inserting an axis along second dimension
    features = features.flatten()[:,np.newaxis]

    # Convert the numpy.ndarray to a Tensor object
    features = Variable(torch.from_numpy(features)).float()
    return features

All the voice samples needed for training are present in the folder `"studio_data"`

In [ ]:
%ls

B17_studio_rev_data.zip  drive/  sample_data/  studio_data/


##**Stage 1**:  Define and train a CNN model on Studio data and deploy the model in the server

---


### a) Extract features of Studio data (4 Marks)

 Load 'Studio data' and extract mfcc features

 **Evaluation Criteria:**

 * Complete the code in the load_data function
 * The function should take path of the folder containing audio samples as input
 * It should return features of all the audio samples present in the specified folder into single array (list of lists or 2-d numpy array) and their respective labels should be returned too

In [ ]:
import os
import re

def load_data(folder_path):
    features = []
    labels = []

    # Iterate through each audio file in the folder
    for filename in os.listdir(folder_path):
        # Check if the file is a WAV file
        if filename.endswith('.wav'):
            # Extract class label from the filename
            match = re.match(r'(\d+)_.*', filename)
            if match:
                label = int(match.group(1))
            else:
                continue  # Skip files that do not match the naming convention

            # Extract features from the audio file using the get_features function
            file_path = os.path.join(folder_path, filename)
            file_features = get_features(file_path)

            # Append the features to the list of features
            features.append(file_features)

            # Append the label to the list of labels
            labels.append(label)

    # Convert the lists of features and labels to numpy arrays
    features = np.array(features)
    labels = np.array(labels)

    return features, labels



Load data from studio_data folder for extracting all features and labels

In [ ]:
studio_recorded_features, studio_recorded_labels = load_data('studio_data/')

In [ ]:
# Print the loaded features and labels
print("Loaded Features:")
print(studio_recorded_features)
print("\nLoaded Labels:")
print(studio_recorded_labels)

Loaded Features:
[[[-8.801408  ]
  [-8.801408  ]
  [-8.801408  ]
  ...
  [ 0.1069589 ]
  [ 0.1069589 ]
  [ 0.1069589 ]]

 [[10.256201  ]
  [10.256201  ]
  [10.256201  ]
  ...
  [-0.9184527 ]
  [-0.9184527 ]
  [-0.9184527 ]]

 [[56.669807  ]
  [56.669807  ]
  [56.669807  ]
  ...
  [ 2.0499554 ]
  [ 2.0499554 ]
  [ 2.0499554 ]]

 ...

 [[25.992481  ]
  [25.992481  ]
  [25.992481  ]
  ...
  [-1.115045  ]
  [-1.115045  ]
  [-1.115045  ]]

 [[56.299694  ]
  [56.299694  ]
  [56.299694  ]
  ...
  [-1.0530806 ]
  [-1.0530806 ]
  [-1.0530806 ]]

 [[ 2.6239715 ]
  [ 2.6239715 ]
  [ 2.6239715 ]
  ...
  [ 0.51757807]
  [ 0.51757807]
  [ 0.51757807]]]

Loaded Labels:
[2 4 5 ... 2 0 2]


Use train_test_split for splitting the train and test data

In [ ]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(studio_recorded_features, studio_recorded_labels, test_size=0.2, random_state=42)

Load the dataset with DataLoader
- Refer to [torch.utils.data.TensorDataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset)
- Refer to [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert training features and labels to PyTorch tensors
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)

# Create a TensorDataset for the training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

# Define batch size
batch_size = 32

# Create a DataLoader for the training dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
# Create a DataLoader for the testing dataset
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)



### b) Define your CNN architecture (4 Marks)

[Hint](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html)

In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Sample Convolution Layer 1
        self.conv1 = nn.Conv1d(in_channels=900, out_channels=400, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(400)
        self.relu1 = nn.ReLU()

        # Sample Maxpool for the Convolutional Layer 1
        self.maxpool1 = nn.MaxPool1d(1)

        # Sample Dropout Layer
        self.dropout = nn.Dropout(p=0.25)

        # Additional Convolutional Layers
        self.conv2 = nn.Conv1d(in_channels=400, out_channels=200, kernel_size=1)
        self.bn2 = nn.BatchNorm1d(200)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(1)

        self.conv3 = nn.Conv1d(in_channels=200, out_channels=100, kernel_size=1)
        self.bn3 = nn.BatchNorm1d(100)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool1d(1)

        # Fully Connected Layer and LogSoftmax
        self.fc = nn.Linear(100, 6)  # 6 output classes
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Convolution Layer 1, Maxpool, and Dropout
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.dropout(out)

        # Convolution Layer 2, Maxpool, and Dropout
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = self.dropout(out)

        # Convolution Layer 3, Maxpool, and Dropout
        out = self.conv3(out)
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.maxpool3(out)
        out = self.dropout(out)

        # Flatten the output of the final pooling layer
        out = out.view(out.size(0), -1)

        # Fully Connected Layer
        out = self.fc(out)

        # Apply LogSoftmax
        out = self.logsoftmax(out)

        return out


In [ ]:
# To run the training on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
model = Net()
model = model.to(device)
print(model)

# Loss function (criterion)
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Net(
  (conv1): Conv1d(900, 400, kernel_size=(1,), stride=(1,))
  (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (maxpool1): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (conv2): Conv1d(400, 200, kernel_size=(1,), stride=(1,))
  (bn2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (maxpool2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(200, 100, kernel_size=(1,), stride=(1,))
  (bn3): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (maxpool3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=100, out_features=6, bias=True)
  (logsoftmax): LogSoftmax(dim=1)
)


### c) Train and classify on the studio_data (3 Marks)

The goal here is to train the Model on voice samples collected in studio data and validate it continuously to calculate the loss and accuracy for the train dataset across each epoch.

Iterate over images in the train_loader and perform the following steps.

1. First, zero out the gradients using zero_grad()

2. Pass the data to the model. Convert the data to GPU before passing data  to the model

3. Calculate the loss using a Loss function

4. Perform Backward pass using backward() to update the weights

5. Optimize and predict by using the torch.max()

6. Calculate the accuracy of the train dataset


In [ ]:
# Number of epochs
num_epochs = 10

# Loop over the dataset for the specified number of epochs
for epoch in range(num_epochs):
    # Set the model to train mode
    model.train()

    # Initialize variables to track loss and accuracy
    running_loss = 0.0
    correct = 0
    total = 0

    # Iterate over the training dataset
    for data in train_loader:
        # Get the inputs and labels
        inputs, labels = data

        # Move inputs and labels to the appropriate device (GPU or CPU)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        # Predict
        _, predicted = torch.max(outputs, 1)

        # Update accuracy statistics
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Update running loss
        running_loss += loss.item()

    # Calculate average loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    # Print epoch statistics
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')


Epoch [1/10], Loss: 1.3838, Accuracy: 47.03%
Epoch [2/10], Loss: 0.9056, Accuracy: 68.71%
Epoch [3/10], Loss: 0.7173, Accuracy: 75.21%
Epoch [4/10], Loss: 0.6507, Accuracy: 76.88%
Epoch [5/10], Loss: 0.5681, Accuracy: 80.43%
Epoch [6/10], Loss: 0.5363, Accuracy: 80.49%
Epoch [7/10], Loss: 0.5012, Accuracy: 82.12%
Epoch [8/10], Loss: 0.4528, Accuracy: 83.41%
Epoch [9/10], Loss: 0.4374, Accuracy: 85.08%
Epoch [10/10], Loss: 0.4014, Accuracy: 86.05%


### d) Testing Evaluation for CNN model (3 Marks)

Evaluate model with the given test data

1. Transform and load the test images.

2. Pass the test data through the model (network) to get the outputs

3. Get the predictions from a maximum value using torch.max

4. Compare with the actual labels and get the count of the correct labels

5. Calculate the accuracy based on the count of correct labels

### **Expected testing accuracy is above 80%**

In [ ]:
# Set the model to evaluation mode
model.eval()

# Initialize variables to track accuracy
correct = 0
total = 0

# Iterate over the test DataLoader
for data in test_loader:
    # Get the inputs and labels
    inputs, labels = data

    # Move inputs and labels to the appropriate device (GPU or CPU)
    inputs = inputs.to(device)
    labels = labels.to(device)

    # Pass the inputs through the model to get the outputs
    outputs = model(inputs)

    # Get the predictions from the maximum value using torch.max
    _, predicted = torch.max(outputs, 1)

    # Compare with the actual labels and get the count of correct labels
    correct += (predicted == labels).sum().item()

    # Update total count of labels
    total += labels.size(0)

# Calculate the accuracy
accuracy = 100 * correct / total

# Print the accuracy
print(f'Testing Accuracy: {accuracy:.2f}%')


Testing Accuracy: 85.18%


### e) Save and download your model (2 Marks)

**Save your model trained on studio data**

* Save the state dictionary of the classifier (use pytorch only), It will be useful in
integrating model to the web application

 [Hint](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

In [ ]:
# Define the file path where you want to save the model
model_path = 'studio_model.pth'

# Save the state dictionary of the model
torch.save(model.state_dict(), model_path)

print(f'Model saved to {model_path}')


Download your trained model using the code below
* Give the path of model file to download through the browser

In [ ]:
from google.colab import files
files.download('studio_model.pth')

### f) Deploy and evaluate your model trained on Studio Data in the server (6 Marks).

(This can be done on the day of the Hackathon once the login username and password provided by the mentors in the lab)

Deploy your model on the server, check the hackathon document (2-Server Access and File transfer For Voice based e-commerce ordering.pdf) for details.

To order product in user interface, go through the document (3-Hackathon_II Application Interface Documentation.pdf) for details.


**Evaluation Criteria: Four consecutive utterances should be predicted correctly by the model**

- There are two stages in the e-commerce ordering application    
    - Ordering Product
    - Selecting the e-commerce platform
- If both the stages are cleared as per the evaluation criteria you will get
complete marks Otherwise, you will see a reduction in the marks

## **Stage 2:** Collect your voice samples and refine the classifier trained on studio_data and Team_data
---

### a) Collect your Team Voice Samples and extract features (6 Marks)

(This can be done on the day of the Hackathon once the login username and password is given by mentors in the lab)

* In order to collect the team data, ensure the server is active (2-Server Access and File transfer For Voice based e-commerce ordering.pdf)

* Refer document "3-Hackathon_II Application Interface Documentation.pdf" for collecting your team voice samples. These will get stored in your server

**Evaluation Criteria:**
* Load 'Team_data' and extract features
* Combine features of team data with the extracted features of studio data
* Split the combined features into train and test data
* Load the dataset with DataLoader

In [ ]:
!mkdir team_data

In [ ]:
# Replace <YOUR_GROUP_ID> with your Username given in the lab
!wget -r -A .wav https://aiml-sandbox1.talentsprint.com/audio_recorder/<YOUR_GROUP_ID>/team_data/ -nH --cut-dirs=100  -P ./team_data

In [ ]:
# YOUR CODE HERE to Load data from teamdata folder for extracting all features and labels

In [ ]:
# Combine the features of all voice samples (studio_data and teamdata)
# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE to split the combined features into train and test data (Hint: Use train_test_split)

In [ ]:
# YOUR CODE HERE to load the dataset with DataLoader

### b) Classify and download the model (6 Marks)

The goal here is to train and test your model on all voice samples collected in studio and team data

**Evaluation Criteria:**
* Refine your classifier (if needed)
* Train your model on the extracted train data
* Test your model on the extracted test data
* Save and download the trained model

### **Expected testing accuracy is above 80%**

In [ ]:
# YOUR CODE HERE for refining your classifier (if needed)

In [ ]:
# YOUR CODE HERE to train your model

# Record loss and accuracy of the train dataset

In [ ]:
# YOUR CODE HERE to test your model

**Save your trained model**

* Save the state dictionary of the classifier (use pytorch only), It will be useful in
integrating model to the web application

 [Hint](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

In [ ]:
### YOUR CODE HERE for saving the CNN model

Download your trained model using the code below
* Give the path of model file to download through the browser

In [ ]:
from google.colab import files
files.download('<model_file_path>')

### c) Deploy and evaluate your model trained on Studio Data + Team Data in the server (6 Marks).

(This can be done on the day of the Hackathon once the login username and password provided by the mentors in the lab)

Deploy your model on the server, check the hackathon document (2-Server Access and File transfer For Voice based e-commerce ordering.pdf) for details.

To order product in user interface, go through the document (3-Hackathon_II Application Interface Documentation.pdf) for details.


**Evaluation Criteria: Four consecutive utterances should be predicted correctly by the model**

- There are two stages in the e-commerce ordering application    
    - Ordering Product
    - Selecting the e-commerce platform
- If both the stages are cleared as per the evaluation criteria you will get
complete marks Otherwise, you will see a reduction in the marks